Cleaning data before upload to database;

In [ ]:
import pandas as pd
df = pd.read_csv('academy_awards.csv', encoding='ISO-8859-1')
df[:5]

In [ ]:
df['Year']=df["Year"].str[0:4]
df['Year'] = df['Year'].astype('int64')



In [ ]:
later_than_2000 = df[ df['Year'] > 2000 ]

In [ ]:
#isin - matches strings in the columns; Filter the Category columns;

award_categories = ["Actor -- Leading Role","Actor -- Supporting Role", "Actress -- Leading Role", "Actress -- Supporting Role"]
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]
nominations[:5]

In [ ]:
#cleaning Won? columns; 
replacements = { "NO": 0, "YES": 1 }
nominations["Won?"] = nominations["Won?"].map(replacements)
nominations["Won"] = nominations["Won?"]
nominations['Won'].head()


In [ ]:
drop_cols = ["Won?","Unnamed: 5", "Unnamed: 6","Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(drop_cols, axis=1)


In [ ]:
#Cleaning additional informaions;

additional_info_one = final_nominations['Additional Info'].str.rstrip("'}")
additional_info_two = additional_info_one.str.split("{'")


In [ ]:
movie_names = additional_info_two.str[0]
characters = additional_info_two.str[1]
#create new columns
final_nominations['movie'] = movie_names
final_nominations['character'] = characters

#remove additional info columns;

final_nominations = final_nominations.drop('Additional Info', axis=1 )
final_nominations.head()


In [ ]:
import sqlite3
conn = sqlite3.connect("nominations.db")
final_nominations.to_sql("nominations", conn, index=False)


In [ ]:
#test;
query_one = "pragma table_info(nominations);"
query_two = "select * from nominations limit 10;"
print(conn.execute(query_one).fetchall())
print(conn.execute(query_two).fetchall())
conn.close()



In [ ]:
import sqlite3
conn = sqlite3.connect("nominations.db")
schema = conn.execute("pragma table_info(nominations);").fetchall()
first_ten = conn.execute("select * from nominations limit 10;").fetchall()

for r in schema:
    print(r)
for row in first_ten:
    print(row)

In [ ]:
# Add hosts name and year: ceremonies;
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

#create_ceremonies = "create table ceremonies (id integer primary key, year integer, host text);"
#conn.execute(create_ceremonies)
insert_query = "insert into ceremonies (Year, Host) values (?,?);"
conn.executemany(insert_query, years_hosts)




In [ ]:
print(conn.execute('select * from ceremonies limit 5;').fetchall())

In [ ]:
#Foreign constraints;
conn.execute('pragma foreign_keys = ON')

In [ ]:
#setting up one-to-many

create_nominations_two = '''
    create table nominations_two( 
        id integer primary key,
        category text,
        nominee text,
        movie text,
        character text,
        won integer,
        ceremony_id integer,
        foreign key( ceremony_id ) references ceremonies(id)
    );
'''

In [ ]:
nom_query='''
select ceremonies.id as ceremony_id, nominations.category as category,
nominations.nominee as nominee, nominations.movie as movie,
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies
on nominations.year == ceremonies.year;
'''



In [ ]:
joined_nominations = conn.execute(nom_query).fetchall()
conn.execute( create_nominations_two )


In [ ]:

insert_nominations_two = '''insert into nominations_two (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''

conn.executemany(insert_nominations_two, joined_nominations)
print(conn.execute("select * from nominations_two limit 5;").fetchall())

In [ ]:
#Deleting and renaming tables;

drop_nominations = ' drop table nominations;'
conn.execute( drop_nominations )

rename_nominations_two = ' alter table nominations_two rename to nominations;'
conn.execute(rename_nominations_two)

In [ ]:
#creating a join table;
create_movies = 'create table movies (id integer primary key,movie text);'
create_actors = ' create table actors (id integer primary key, actor text);'

create_movies_actors = '''create table movies_actors (id INTEGER PRIMARY KEY,
movie_id INTEGER references movies(id), actor_id INTEGER references actors(id));
'''

conn.execute(create_movies)
conn.execute( create_actors)
conn.execute( create_movies_actors)

In [ ]:
## add movies and actors;



insert_movies = "insert into movies (movie) select distinct movie from nominations;"
insert_actors = "insert into actors (actor) select distinct nominee from nominations;"
conn.execute(insert_movies)
conn.execute(insert_actors)

print(conn.execute("select * from movies limit 5;").fetchall())
print('-------')
print(conn.execute("select * from actors limit 5;").fetchall())

In [ ]:
#join tables;


pairs_query = "select movie,nominee from nominations;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()

join_table_insert = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?),(select id from actors where actor == ?));"
conn.executemany(join_table_insert,movie_actor_pairs)

print(conn.execute("select * from movies_actors limit 5;").fetchall())

